In [1]:
# Import libraries

import os
import requests
from io import StringIO

import psycopg2
from psycopg2 import sql

# Data Manipulation
import pandas as pd
import geopandas as gpd

# Getting .env information (API keys, etc.)
from dotenv import load_dotenv

In [2]:
# Get REDCap token

load_dotenv()

redCap_token_signUp = os.getenv('REDCAP_TOKEN_SIGNUP') # Survey Token
redCap_token_report = os.getenv('REDCAP_TOKEN_REPORT') # Report Token

# Get Database credentials

creds = [os.getenv('DB_NAME'),
         os.getenv('DB_USER'),
         os.getenv('DB_PASS'),
         os.getenv('DB_PORT'),
         os.getenv('DB_HOST')
        ]

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))

In [91]:
# filterLogic Redcap API
# For how to do this - https://education.arcus.chop.edu/redcap-api/
# Redcap logic guide - https://cctsi.cuanschutz.edu/docs/librariesprovider28/redcap/redcap-logic-guide.pdf?sfvrsn=258e94ba_2
# Info from PyCap - https://redcap-tools.github.io/PyCap/api_reference/records/

### Select by record_id

# They must be strings before below

# record_ids = ['1', '2', '3', '4']
record_ids = [2,3,4,1] # Intentionally mixing up order

record_id_strs = [str(record_id) for record_id in record_ids]

filterLogic_str = '[record_id]=' + ' OR [record_id]='.join(record_id_strs)

### field_names

field_names = 'phone, record_id'

In [92]:
data = {
    'token': redCap_token_signUp,
    'content': 'record',
    'fields' : field_names,
    'action': 'export',
    'format': 'csv',
    'type': 'flat',
    'csvDelimiter': '',
    'rawOrLabel': 'raw',
    'rawOrLabelHeaders': 'raw',
    'exportCheckboxLabel': 'false',
    'exportSurveyFields': 'false',
    'exportDataAccessGroups': 'false',
    'returnFormat': 'csv',
    'filterLogic': filterLogic_str  
}
r = requests.post('https://redcap.ahc.umn.edu/api/',data=data)

In [99]:
print('HTTP Status: ' + str(r.status_code))
df = pd.read_csv(StringIO(r.text))

# df

HTTP Status: 200


In [100]:
# Sort df by record_ids from above

sorted_df = df.set_index('record_id').loc[record_ids]

# sorted_df

In [101]:
# get phone numbers as a list

phone_numbers = list(sorted_df.phone)

# phone_numbers